In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Model Optimizer

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/model-optimizer/intro_model_optimizer.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fmodel-optimizer%2Fintro_model_optimizer.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/model-optimizer/intro_model_optimizer.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/model-optimizer/intro_model_optimizer.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>



<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/model-optimizer/intro_model_optimizer.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/model-optimizer/intro_model_optimizer.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/model-optimizer/intro_model_optimizer.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/model-optimizer/intro_model_optimizer.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/model-optimizer/intro_model_optimizer.ipynb">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author |
| --- |
| [Nardos Alemu](https://github.com/nardosalemu) |

## Overview

Model Optimizer intelligently routes user queries across models (Pro, Flash, etc.) for resource optimization. This notebook provides an introduction on how to use Model Optimizer with the Google Gen AI SDK.

Learn more about [Model Optimizer](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/vertex-ai-model-optimizer).

### Objectives

This tutorial shows how to:

-   Send prompt queries to Model Optimizer.
-   Configure routing preferences to optimize quality or cost.
-   Utilize Model Optimizer for single-turn and multi-turn conversations.
-   Execute function calls through Model Optimizer.

### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Install Google Gen AI SDK

Learn more about [Google Gen AI SDK](https://cloud.google.com/vertex-ai/generative-ai/docs/sdks/overview).

In [ ]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries

In [ ]:
from IPython.display import Markdown, display
from google.genai.types import (
    FeatureSelectionPreference,
    FunctionDeclaration,
    GenerateContentConfig,
    ModelSelectionConfig,
    Part,
    Tool,
)

### Load Model & set System Instruction

`model-optimizer-exp-04-09` is the model used for Model Optimizer.

In [ ]:
MODEL_ID = "model-optimizer-exp-04-09"
system_instruction = ""  # @param {'type': 'string'}

### Choose Routing Option

Model Optimizer intelligently routes user queries based on user-defined cost and quality preferences. This dynamic routing capability enables users to optimize their interactions according to their specific needs.

In [ ]:
model_selection_config = ModelSelectionConfig(
    feature_selection_preference=FeatureSelectionPreference.BALANCED  # Options: PRIORITIZE_QUALITY, BALANCED, PRIORITIZE_COST
)

### Send a request

The following function is defined to send a request to the model. `generate_content` is the method used to generate response. The method supports streaming with `stream=True`. The result has the same type as the non-streaming case, but you can iterate over the response chunks as they become available.

In [ ]:
def generate_content(prompt, stream: bool = False) -> None:
    if stream:
        output_text = ""
        markdown_display_area = display(Markdown(output_text), display_id=True)

        for chunk in client.models.generate_content_stream(
            model=MODEL_ID,
            contents=prompt,
            config=GenerateContentConfig(
                system_instruction=system_instruction,
                model_selection_config=model_selection_config,
            ),
        ):
            output_text += chunk.text
            markdown_display_area.update(Markdown(output_text))
    else:
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt,
            config=GenerateContentConfig(
                system_instruction=system_instruction,
                model_selection_config=model_selection_config,
            ),
        )
        display(Markdown(response.text))

#### Single-turn query

A single-turn query is the most basic type of interaction with the model, consisting of a single user prompt and a single model response. It's suitable for simple requests where context is not essential.

In [ ]:
prompt = "Write a poem"  # @param {'type': 'string'}
generate_content(prompt, stream=True)

#### Multi-turn queries

Multi-turn queries involve ongoing conversations with the model, where the model retains context from previous turns to generate relevant responses. These allow for more complex interactions and the development of conversational flows.

There are two ways to structure multi-turn queries:

- **List of text prompts:** A simple list of strings, where each string represents a turn in the conversation. This format is suitable for basic multi-turn scenarios.
- **List of `Content` objects:** A more structured format using `Content` objects, where you can explicitly define the role (user or model) and parts of each turn. This format provides greater control and clarity, especially when dealing with complex conversations.

In [ ]:
prompt = [
    "What is x multiplied by 2?",
    "x = 42",
]
generate_content(prompt)

#### Function-call queries

Model Optimizer also supports calls to functions to perform actions and retrieve information. You define the function's signature (name and parameters), and the model can generate a call to that function with appropriate arguments when it determines that doing so would be helpful. In this example, we define a simple function `get_current_weather` and demonstrate how the model can call it.

In [ ]:
def get_current_weather(location: str, unit: str | None = "celsius") -> dict:
    """Gets weather in the specified location.

    Args:
        location: The location for which to get the weather.
        unit: Temperature unit. Can be Celsius or Fahrenheit. Default: Celsius.

    Returns:
        The weather information as a dict.
    """
    return {
        "location": location,
        "unit": unit,
        "weather": "Super nice, but maybe a bit hot.",
    }


_REQUEST_FUNCTION_PARAMETER_SCHEMA_STRUCT = {
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
        },
        "unit": {
            "type": "string",
            "enum": [
                "celsius",
                "fahrenheit",
            ],
        },
    },
    "required": ["location"],
}

_REQUEST_FUNCTION_RESPONSE_SCHEMA_STRUCT = {
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
        },
        "unit": {
            "type": "string",
            "enum": [
                "celsius",
                "fahrenheit",
            ],
        },
        "weather": {
            "type": "string",
        },
    },
}

get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters=_REQUEST_FUNCTION_PARAMETER_SCHEMA_STRUCT,
    response=_REQUEST_FUNCTION_RESPONSE_SCHEMA_STRUCT,
)


prompt = "What is the weather like in Boston?"

weather_tool = Tool(function_declarations=[get_current_weather_func])

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        tools=[weather_tool],
        model_selection_config=model_selection_config,
        system_instruction=system_instruction,
    ),
)

print("Called function: ", response.function_calls)

To preview more details:

In [ ]:
function_map = {"get_current_weather": get_current_weather}

function_response_parts = []
for function_call in response.function_calls:
    function = function_map[function_call.name]
    function_result = function(**function_call.args)
    function_response_parts.append(
        Part.from_function_response(
            name=function_call.name,
            response=function_result,
        )
    )

print(function_response_parts)